### Goals

1. Write a program that can create a list of all nxn latin squares, call it Ln. This is probably only computationally feasible up to around 6x6.
2. Write a function that "standardizes" a latin square by permuting the symbols such that the first row of the square is in order, 1, 2, 3, ..., n. Call this function F()
3. Work through the following algorithm, starting at 4x4 latin squares. 5x5 should also be possible, 6x6 maybe not. 
    - Select a latin square from Ln, call it Sj.
    - Make a list of all latin squares that can be obtained from Sj by permuting the rows and columns, call this Aj.
    - Count how many squares there are and how many of them are equal to Sj.
    - Remove the squares in Aj from Ln and repeat a-c until there are none left. Count how many of these different "classes" there are and how big each class is.

All of this could also be adjusted to standardize all the squares by F().

### Misc Functions

In [5]:
def print_square(square):
    '''
    Print squre row by row for better readability.
    '''
    for row in square:
        print(row)
    print()

### 1. Generating Latin Squares

In [ ]:
from itertools import permutations

def Ln(n: int) -> list:
    '''
    Generates all Latin squares of order n.
    '''
    all_rows = list(permutations(range(1,n+1)))
    squares = []

    def backtrack(square, depth):
        if depth == n:
            squares.append([list(row) for row in square])
            return
        
        for row in all_rows:
            if all(row[col] not in used_in_col[col] for col in range(n)):
                square.append(row)
                for col in range(n):
                    used_in_col[col].add(row[col])

                backtrack(square, depth + 1)

                for col in range(n):
                    used_in_col[col].remove(row[col])
                square.pop()
    used_in_col = [set() for _ in range(n)]

    backtrack([],0)
    return squares

def Ln_reduced(n: int,) -> list:
    '''
    Generates all reduced Latin squares of order n,
    fixing the first row to [1, 2, ..., n].
    This removes symmetries due to row permutations.
    '''
    all_rows = list(permutations(range(1, n + 1)))
    squares = []

    def backtrack(square, depth):
        if depth == n:
            squares.append(square[:])
            return

        for row in all_rows:
            if all(row[col] not in used_in_col[col] for col in range(n)):
                square.append(row)
                for col in range(n):
                    used_in_col[col].add(row[col])

                backtrack(square, depth + 1)

                for col in range(n):
                    used_in_col[col].remove(row[col])
                square.pop()

    first_row = tuple(range(1, n + 1))
    used_in_col = [set() for _ in range(n)]
    for col in range(n):
        used_in_col[col].add(first_row[col])  # track first row values
        
    backtrack([first_row], 1)
    return squares
    

#### 1. Test Cases

In [77]:
three_by_three_reduced = Ln_reduced(3) 
three_by_three = Ln(3)
print(f"Total latin square of order 3: {len(three_by_three)}")
print(f"Reduced latin square of order 3: {len(three_by_three_reduced)}")

for square in three_by_three:
    #print_square(square)
    None

Total latin square of order 3: 12
Reduced latin square of order 3: 2


In [78]:
four_by_four = Ln(4)
four_by_four_reduced = Ln_reduced(4)
print(f"Total latin square of order 4: {len(four_by_four)}")
print(f"Reduced latin square of order 4: {len(four_by_four_reduced)}")
for square in four_by_four:
    #print_square(square)
    None
# 576 squares total
# 24 reduced

Total latin square of order 4: 576
Reduced latin square of order 4: 24


In [79]:
five_by_five = Ln(5)
five_by_five_reduced = Ln_reduced(5)
print(f"Total latin square of order 5: {len(five_by_five)}")
print(f"Reduced latin square of order 5: {len(five_by_five_reduced)}")
for square in five_by_five:
    #print_square(square)
    None
# 161280 Squares
# 1344 total

Total latin square of order 5: 161280
Reduced latin square of order 5: 1344


In [ ]:
six_by_six = Ln(6)
six_by_six_reduced = Ln_reduced(6)
print(f"Total latin square of order 6: {len(six_by_six)}")
print(f"Total latin square of order 6: {len(six_by_six_reduced)}")
for square in six_by_six:
    #print_square(square)
    None

### 2. "Standardize" Squares (Symmetry Reduction)

In [ ]:
def Fn(square: list) -> list:
    '''
    Standardize a Latin square so that the first row is [1, 2, ..., n].
    '''
    first_row = square[0]

    # Creates a mapping dict for standardization of square
    row_mapping = {val: i + 1 for i, val in enumerate(first_row)}

    # Apply the mapping to the entire square
    standardized_square = [[row_mapping[val] for val in row] for row in square]

    return standardized_square

##### 2. Test Cases

In [ ]:
square1 = [
    [3, 1, 2],
    [2, 3, 1],
    [1, 2, 3]
]

''' 
Expected output:
    [1, 2, 3],
    [3, 1, 2], 
    [2, 3, 1] ] 
'''

square2 = [
    [4, 1, 3, 2],
    [2, 4, 1, 3],
    [1, 3, 2, 4],
    [3, 2, 4, 1]
]

''' 
Expected output:
    [1, 2, 3, 4],
    [4, 1, 2, 3],
    [2, 3, 4, 1],
    [3, 4, 1, 2] 
'''

square3 = [
    [1, 2, 3, 4, 5],
    [2, 3, 4, 5, 1],
    [3, 4, 5, 1, 2],
    [4, 5, 1, 2, 3],
    [5, 1, 2, 3, 4]
]
'''
Already "standardized," expected output = input
'''

for i in [square1, square2, square3]:
    result = Fn(i)
    print_square(result)

[1, 2, 3]
[3, 1, 2]
[2, 3, 1]

[1, 2, 3, 4]
[4, 1, 2, 3]
[2, 3, 4, 1]
[3, 4, 1, 2]

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 1]
[3, 4, 5, 1, 2]
[4, 5, 1, 2, 3]
[5, 1, 2, 3, 4]



### 3. Main Algorithm

In [ ]:
def main():
    '''
    Selects a latin square from Ln, Sj.
    Permutes the rows and columns, creates list of Sj's orbits & Stabalizers, Aj.
    Count how many squares in Aj are stabalizers.
    Remove stabalizers ffrom Ln and repeat until Ln is empty. 
    Count how many of these different "classes" there are and how big each class is.
    '''
    return 1

if __name__ == "__main__":
     main()

1
